# Progama brasileiro de imunização - COVID 19

## Objetivos

Transformar os dados brutos dos registro de vacinação para um modelo analítico, disponibilizando para consulta pública.

Utilizaremos os seguintes conjuntos de dados:
* [Registros de Vacinação COVID19 do Ministério da Saúde](https://dados.gov.br/dataset/covid-19-vacinacao/resource/ef3bd0b8-b605-474b-9ae5-c97390c197a8?inner_span=True)
    * O processamento será feito por Unidade Federativa, para fins de otimização

**Os dados serão extraídos dos arquivos CSV fornecidos pelo Ministério da Saúde, e carregados em um conjunto de tabelas analíticas, seguindo o modelo [Star Schema](https://en.wikipedia.org/wiki/Star_schema).**.

## Enviroment setup

In [ ]:
# Bibliotecas utilizadas
import configparserimport pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType


In [ ]:
# Read config file
config = configparser.ConfigParser()
config.read_file(open('dl.cfg'))

INPUT_DATA = config['LOCAL']['INPUT_DATA']
INPUT_DATA_VACCINES = config['LOCAL']['INPUT_DATA_VACCINES']
OUTPUT_DATA = config['LOCAL']['OUTPUT_DATA']
DATA_COLUMNS = config['COMMON']['DATA_COLUMNS']

In [ ]:
def write_parquet(df, parquet_name):
    parquet_path = OUTPUT_DATA + f'{parquet_name}.parquet'
    df.write.mode("overwrite").parquet(parquet_path)


In [ ]:
def read_parquet(parquet_name):
    parquet_path = OUTPUT_DATA + f'{parquet_name}.parquet'
    return spark.read.parquet(parquet_path)

In [ ]:
def check_nulls(df, columns_list, expected_value):
    df.createOrReplaceTempView("viewcheck")
    sql_check = f"SELECT COUNT(*) FROM viewcheck WHERE 1 <> 1 {''.join([' OR ' + c + ' IS NULL ' for c in columns_list])}"
    
    dfcheck = spark.sql(sql_check)
    
    value_check = dfcheck.collect()[0][0]
    
    return value_check == expected_value

In [ ]:
def check_has_content(df):
    return df.count() > 0

In [ ]:
# Spark session
spark = SparkSession \
        .builder\
        .appName("Brasil - Programa de imunização - COVID-19")\
        .getOrCreate()

In [ ]:
dados_imunizacao = spark.read.csv(INPUT_DATA_VACCINES, sep=';', header=True)

dados_imunizacao.printSchema()

## Step 2: Explore and Assess the Data
In this step we need:
* Explore the data to identify data quality issues, like missing values, duplicate data, etc.
* Document steps necessary to clean the data

In [ ]:
# Obter a lista de colunas que serão utilizadas
col_names = pd.read_json(DATA_COLUMNS, typ='series')
colunas_utilizadas = col_names.index
colunas_utilizadas

In [ ]:
# Obter as colunas não utilizadas
colunas_naoutilizadas = list(set(dados_imunizacao.columns) - set(colunas_utilizadas))

colunas_naoutilizadas

In [ ]:
# Excluir do dataset as colunas não utilizadas
dados_imunizacao = dados_imunizacao.drop(*colunas_naoutilizadas)
dados_imunizacao.printSchema()

In [ ]:
# Trocar os valores Nulos
dados_imunizacao = dados_imunizacao.fillna(\
    {\
        'vacina_categoria_codigo': 0, \
        'vacina_categoria_nome': 'N/A', \
        'vacina_grupoatendimento_nome': 'N/A', \
        'paciente_enumsexobiologico': 'N/A',\
        'paciente_endereco_nmmunicipio': 'N/A', \
        'paciente_endereco_nmpais': 'N/A', \
        'paciente_endereco_uf': 'N/A', \
        'estalecimento_nofantasia': 'N/A'
    })

In [ ]:
dados_imunizacao.printSchema()

In [ ]:
# Cria o arquivo parquet com os dados temporários
write_parquet(dados_imunizacao, 'dados_imunizacao')

dados_imunizacao = read_parquet('dados_imunizacao')

## Step 3: Define the Data Model
_Map out the conceptual data model and explain why you chose that model_

The data model is a star schema consisting of 5 Dimensions table and 1 Fact table:
  * Dimensions tables:
      * vaccines table: Vaccines and suppliers
      * health_institution table: Hospitals, Nursing home, Clinics 
      * category table: Priority groups
      * population_group table: Demograph group (professions, age group, ethnicity)
      * patient table: Demograph data (age, city, gender)
  * Fact table:
      * immunization table: Dimensions, First | second dose, date

![ER Data Model - Star Scheme](./docs/er-model-star.jpg)

_List the steps necessary to pipeline the data into the chosen data model_
* ETL starts the enviroment setup: imports, read config file, def functions and create Spark Session
* ETL script takes source data (Brazilian Government' dataset COVID-19 population immunization program)
* Raw data is read into dataframe and cleaned (remove unused columns, fill nulls) 
* For each dimension and fact table 
	* Create a temporary view table
	* Read data to new dataframe
    * Check data quality: key columns don't have nulls, each table has content
	* Create id/indexes (if necessary)
	* Write parquet files

## Step 4: Run ETL to Model the Data
* Create the data pipelines and the data model
* Include a data dictionary
* Run data quality checks to ensure the pipeline ran as expected
	* Integrity constraints on the relational database (e.g., unique key, data type, etc.)
	* Unit tests for the scripts to ensure they are doing the right thing
	* Source/count checks to ensure completeness

In [ ]:
# Criar tabela Vacinas
dados_imunizacao.createOrReplaceTempView("vacinas_df")
vacinas_df = spark.sql("""
    SELECT  DISTINCT vacina_codigo AS codigo, 
                     vacina_nome AS descricao, 
                     vacina_fabricante_nome AS fabricante
    FROM vacinas_df
    ORDER BY codigo, fabricante, descricao
""")

vacinas_df.printSchema()
vacinas_df.show()

In [ ]:
# Verifica valores nulos ou se há valores, antes de salvar a tabela
if not check_nulls(vacinas_df, ['codigo', 'descricao', 'fabricante'], 0): raise Exception('Null: Vacinas')
if not check_has_content(vacinas_df): raise Exception('No content: Vacinas')write_parquet(vacinas_df, 'vacinas')

In [ ]:
# Criar tabela Estabelecimentos
dados_imunizacao.createOrReplaceTempView("estabelecimento_df")
estabelecimento_df = spark.sql("""
    SELECT DISTINCT estabelecimento_valor AS codigo,
                    estalecimento_nofantasia AS descricao,
                    estabelecimento_razaosocial AS razaosocial,
                    estabelecimento_uf AS uf,
                    estabelecimento_municipio_nome AS municipio
    FROM estabelecimento_df
    ORDER BY uf, municipio, razaosocial, descricao
""")

estabelecimento_df.printSchema()
estabelecimento_df.show()

In [ ]:
# Verifica valores nulos ou se há valores, antes de salvar a tabela
if not check_nulls(estabelecimento_df, ['codigo', 'descricao', 'razaosocial', 'uf', 'municipio'], 0): raise Exception('Null: Estabelecimentos')
if not check_has_content(estabelecimento_df): raise Exception('No content: Estabelecimentos')write_parquet(estabelecimento_df, 'estabelecimentos')

In [ ]:
# Criar tabela Categorias
dados_imunizacao.createOrReplaceTempView("categorias_df")
categorias_df = spark.sql("""
    SELECT DISTINCT vacina_categoria_codigo AS codigo,
                    vacina_categoria_nome AS descricao
            FROM categorias_df
            ORDER BY codigo
""")

categorias_df.printSchema()
categorias_df.show()

In [ ]:
# Verifica valores nulos ou se há valores, antes de salvar a tabela
if not check_nulls(categorias_df, ['codigo', 'descricao'], 0): raise Exception('Null: Categorias')
if not check_has_content(categorias_df): raise Exception('No content: Categorias')write_parquet(categorias_df, 'categorias')

In [ ]:
# Criar tabela Grupos de Atendimento
dados_imunizacao.createOrReplaceTempView("grupos_atendimento_df")
grupos_atendimento_df = spark.sql("""
    SELECT DISTINCT vacina_grupoatendimento_codigo AS codigo,
                    vacina_grupoatendimento_nome AS descricao
            FROM grupos_atendimento_df
        ORDER BY codigo
""")

grupos_atendimento_df.printSchema()
grupos_atendimento_df.show()

In [ ]:
# Verifica valores nulos ou se há valores, antes de salvar a tabela
if not check_nulls(grupos_atendimento_df, ['codigo', 'descricao'], 0): raise Exception('Null: Grupos de atendimento')
if not check_has_content(grupos_atendimento_df): raise Exception('No content: Grupos de atendimento')write_parquet(grupos_atendimento_df, 'grupos')

In [ ]:
# Criar tabela Pacientes
dados_imunizacao.createOrReplaceTempView("pacientes_df")
pacientes_df = spark.sql("""
    SELECT DISTINCT paciente_id AS id,
                    paciente_idade AS idade,
                    paciente_datanascimento AS datanascimento,
                    paciente_enumsexobiologico AS sexo,
                    paciente_endereco_nmpais AS pais,
                    paciente_endereco_uf AS uf,
                    paciente_endereco_nmmunicipio AS municipio
            FROM pacientes_df
            WHERE paciente_id IS NOT NULL
            ORDER BY id
""")

pacientes_df.printSchema()
pacientes_df.show()

In [ ]:
# Verifica valores nulos ou se há valores, antes de salvar a tabela
if not check_nulls(pacientes_df, ['id', 'idade', 'datanascimento', 'sexo', 'pais', 'uf', 'municipio'], 0): raise Exception('Null: Pacientes')
if not check_has_content(pacientes_df): raise Exception('No content: Pacientes')write_parquet(pacientes_df, 'pacientes')

In [ ]:
# Create Patient table and write parquet files
dados_imunizacao.createOrReplaceTempView("vacinacao")
vacinacao_df = spark.sql("""
    SELECT DISTINCT paciente_id AS paciente_id,
            estabelecimento_valor AS estabelecimento,
            vacina_categoria_codigo AS categoria,
            vacina_grupoatendimento_codigo AS grupoatendimento,
            vacina_codigo AS vacina,
            vacina_descricao_dose AS dose,
            vacina_dataaplicacao AS dataaplicacao
        FROM vacinacao 
        WHERE paciente_id IS NOT NULL
        ORDER BY dataaplicacao
""")

vacinacao_df.printSchema()

In [ ]:
# Check data quality
if not check_nulls(vacinacao_df, \
        ['paciente_id', \
        'estabelecimento', \
        'categoria', \
        'grupoatendimento', \
        'vacina', \
        'dose', \
        'dataaplicacao'], 0): raise Exception('Null: Vacinacao')
if not check_has_content(vacinacao_df): raise Exception('No content: Vacinacao')

In [ ]:
# Write parquet file and get back to Spark:
write_parquet(vacinacao_df, 'vacinacao')


In [ ]:
vacinacao_df.show()

### Data dictionary

[Dict](./data-dictionary.json)
```
{
  "vaccines": {
    "description": "Vaccines and suppliers",
    "fields": {
      "id": "Identity field",
      "name": "Vaccine name",
      "supplier": "Laboratory/supplier"
    }
  },
  "health_institution": {
    "description": "Hospitals, Nursing home, Clinics",
    "fields": {
      "id": "Identity field",
      "name": "Institution name",
      "organization": "",
      "state": "Brazilian State",
      "city": "City name"
    }
  },
  "category": {
    "description": "Priority groups",
    "fields": {
      "id": "Identity field",
      "name": "Group name"
    }
  },
  "population_group": {
    "description": "Demograph group (professions, age group, ethnicity)",
    "fields": {
      "id": "Identity field",
      "name": "Group name"
    }
  },
  "patient": {
    "description": "Demograph data (age, city, gender)",
    "fields": {
      "id": "Identity field",
      "age": "Patient age (years)",
      "birthdate": "Birth date",
      "gender": "Male/Female",
      "country": "Country (Brazil)",
      "state": "Brazilian State",
      "city": "City name"
    }
  },
  "imunization": {
    "description": "Dimensions, First | second dose, date",
    "fields": {
      "patient_id": "(FK) Patient Identity",
      "health_institution_id": "(FK) Health institution Identity",
      "category_id": "(FK) Category group Identity",
      "population_group_id": "(FK) Population group Identity",
      "vaccines_id": "(FK) Vaccine Identity",
      "vaccines_dose": "First/Second dose",
      "jab_date": "Vaccine date"
    }
  }
}
```

## Step 5: Complete Project Write Up
Tools:
* Python
* Pandas
* Spark

These tools/technologies are apropriated to manipulate large dataset, processing in paralellized clusters

**ETL script should be run weekly basis**, or whenever the .gov.br update the datasets

How the script would approach the problem differently under the following scenarios:
* If the data was increased by 100x: _Use Spark Clustered to parallel the data load_
* If the pipelines were run on a daily basis by 7am: _Refactor the script to process only new informations. The [.gov.br API](https://dados.gov.br/dataset/covid-19-vacinacao/resource/97a8fbcf-941f-4d2e-91ba-dd467d5bdeac?inner_span=True) could be used to request the delta information_
* If the database needed to be accessed by 100+ people: _Store the parquet files on a cloud data lake_

### Sample queries

In [ ]:
vacinacao_df = read_parquet('vacinacao')estabelecimento_df = read_parquet('estabelecimentos')
vacinas_df = read_parquet('vacinas')

# Obter estabelecimentos por aplicação
vacinas_df.createOrReplaceTempView("vdf")
estabelecimento_df.createOrReplaceTempView("hidf")
vacinacao_df.createOrReplaceTempView("idf")

estabelecimentos_total = spark.sql("""
    SELECT 
        CASE
            WHEN (GROUPING(hidf.descricao) = 1) THEN '# Total Estabelecimento'
            ELSE hidf.descricao
        END AS Estabelecimento,
        CASE
            WHEN (GROUPING(vdf.descricao) = 1) THEN '# Total Vacina'
            ELSE vdf.descricao
        END AS Vacina,
        COUNT(*) AS TOTAL
    FROM idf 
    INNER JOIN hidf ON idf.estabelecimento = hidf.codigo
    INNER JOIN vdf ON idf.vacina = vdf.codigo
    GROUP BY CUBE(hidf.descricao, vdf.descricao)
    ORDER BY Estabelecimento, TOTAL DESC, Vacina
""").toPandas()

estabelecimentos_total.head()